# Order Paramater with Respect to Bilayer Normal for Cytoplasmic Lipids grouped across from Lo or Ld

In [2]:
from __future__ import print_function
%matplotlib inline
import mdtraj as md
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import PatchCollection
from matplotlib.patches import Polygon
#import scipy.cluster.hierarchy
import scipy.spatial
import math

In [ ]:
traj=md.load('a_pttwo.xtc',top='a_pttwo_19ns.gro',stride=1)

In [ ]:
traj.n_frames

In [4]:
name = '1'

In [5]:
topology=traj.topology
DBPCs=topology.select('resid 2112 to 2990') #selected only top leaflet DBPC 
lipidhead=topology.select('resname POPE')
DBPClength=int(52)
numberoflipids=889

In [8]:
#x,y coordinates of all for finding neighbors to determine local midplane z-value
# here, best to use xy radius of neighbors, since Voronoi across 2 leaflets could have complications

lipidheadxy=list([] for _ in xrange(traj.n_frames))
lipidheadxyz=list([] for _ in xrange(traj.n_frames))
for nn in range(traj.n_frames):
    for i in range(len(lipidhead)):
        lipidheadi=lipidhead[i]
        lipidheadxy[nn].append(traj.xyz[nn][lipidheadi][0:2:1])
        lipidheadxyz[nn].append(traj.xyz[nn][lipidheadi][0:3:1])


# Random Methods

In [9]:
def find_plane(cord1,cord2,cord3):
    v1 = cord3-cord1
    v2 = cord3-cord2
    cp=np.cross(v1,v2)
    d = cp[0]**2 + cp[1]**2 + cp[2]**2
    x,y,z = cp/(d**.5)
    return [x,y,z]

In [10]:
def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]

In [11]:
def local_vector(cord1,cord2):
    vector = [cord1[0]-cord2[0],cord1[1]-cord2[1],cord1[2]-cord2[2]]
    d=(cord1[0]-cord2[0])**2 + (cord1[1]-cord2[1])**2 + (cord1[2]-cord2[2])**2
    vector=vector/(d**.5)
    return vector
    

In [12]:
def find_plane(cord1,cord2,cord3):
    v1 = cord3-cord1
    v2 = cord3-cord2
    cp=np.cross(v1,v2)
    d = cp[0]**2 + cp[1]**2 + cp[2]**2
    x,y,z = cp/(d**.5)
    return [x,y,z]

In [13]:
def magvect(v):
    magnitude = np.sqrt(v[0]*v[0] + v[1]*v[1] + v[2]*v[2])
    return magnitude

In [14]:
def cos_angle(v1,v2):
    cos = (np.dot(v1,v2)/((magvect(v1)*magvect(v2))))
    return cos

In [15]:
def perpendicular_vector(v):
    if v[1] == 0 and v[2] == 0:
        if v[0] == 0:
            raise ValueError('zero vector')
        else:
            return np.cross(v, [0, 1, 0])
    return np.cross(v, [1, 0, 0])

# Find Membrane Normal

In [16]:
topology=traj.topology
allhead=topology.select('name P8 or name C9 or name O1')

In [17]:
resnames=[atom.residue.name for atom in topology.atoms]
residue=[atom.residue for atom in topology.atoms]

In [18]:
#x,y coordinates of all for finding neighbors to determine local midplane z-value
# here, best to use xy radius of neighbors, since Voronoi across 2 leaflets could have complications

allheadxy=list([] for _ in xrange(traj.n_frames))
allheadz=list([] for _ in xrange(traj.n_frames))
for nn in range(traj.n_frames):
    for i in range(len(allhead)):
        allheadi=allhead[i]
        allheadxy[nn].append(traj.xyz[nn][allheadi][0:2:1])
        allheadz[nn].append(traj.xyz[nn][allheadi][2])


In [19]:
# mdtraj can't handle neighbors for gro (rather than xtc) inputs, so do it myself
#cutoffsq = 2.4**2; #square of maximum xy distance for neighbors
cutoffsq=1.0**2
neigh=list([] for _ in xrange(traj.n_frames))
for nn in range(traj.n_frames):
    neigh[nn]=list([] for _ in xrange(len(allhead)))
    for i in range(len(allheadxy[nn])):
        xyi=allheadxy[nn][i]
        for j in range(len(allhead)-i-1): #not self, but will add for midplane finding
            xyj=allheadxy[nn][j+i+1]
            distsq=(xyi[0]-xyj[0])**2 + (xyi[1]-xyj[1])**2
            if (distsq < cutoffsq):
                neigh[nn][i].append(j+i+1)
                neigh[nn][j+i+1].append(i)
  

In [20]:
#new leaflet id method based on tilt angles; PO4-C4A or ROH-C1 (both +6 beads)
num_head=len(allhead)
tiltvectors=list([] for _ in xrange(traj.n_frames))

for nn in range(traj.n_frames):
    tiltvectors[nn]=list([] for _ in xrange(num_head)) #store vector roh-c1 or po4-c4a
    
    for i in range(num_head):
        tiltvectors[nn][i]=traj.xyz[nn][allhead[i]]-traj.xyz[nn][allhead[i]+6]


In [21]:
# must fix periodic boundary condition errors in tiltvectors, then calculate tiltangle
norms=list([] for _ in xrange(traj.n_frames))
for nn in range(traj.n_frames):
    norms[nn]=list([] for _ in xrange(len(allhead)))
    halfx=0.5*traj.unitcell_lengths[nn][0]
    halfy=0.5*traj.unitcell_lengths[nn][1]
    halfz=0.5*traj.unitcell_lengths[nn][2]
    for i in range(len(allhead)):
        norms[nn][i]=np.linalg.norm(tiltvectors[nn][i])
        if (norms[nn][i] > halfz):
            if (np.abs(tiltvectors[nn][i][0]) > halfx):
                if (tiltvectors[nn][i][0]>0): 
                    tiltvectors[nn][i][0]=tiltvectors[nn][i][0]-2*halfx
                else:
                    tiltvectors[nn][i][0]=tiltvectors[nn][i][0]+2*halfx
            if (np.abs(tiltvectors[nn][i][1]) > halfy):
                if (tiltvectors[nn][i][1]>0): 
                    tiltvectors[nn][i][1]=tiltvectors[nn][i][1]-2*halfy
                else:
                    tiltvectors[nn][i][1]=tiltvectors[nn][i][1]+2*halfy
            if (np.abs(tiltvectors[nn][i][2]) > halfz):
                if (tiltvectors[nn][i][2]>0): 
                    tiltvectors[nn][i][2]=tiltvectors[nn][i][2]-2*halfz
                else:
                    tiltvectors[nn][i][2]=tiltvectors[nn][i][2]+2*halfz

    for i in range(len(allhead)):
        norms[nn][i]=np.linalg.norm(tiltvectors[nn][i])


In [22]:
# use neigh to find local average tilt vector, outliers are not in a leaflet
# if pointing neg in z, flip in x,y,z for making average in outer leaflet
localvector=list([] for _ in xrange(traj.n_frames))
for nn in range(traj.n_frames):
    localvector[nn]=list([] for _ in xrange(len(allhead)))
    for i in range(len(allhead)):
        localvectorsx=[]
        localvectorsy=[]
        localvectorsz=[]
        sgn=np.sign(tiltvectors[nn][i][2])
        localvectorsx.append(sgn*tiltvectors[nn][i][0]) #include self
        localvectorsy.append(sgn*tiltvectors[nn][i][1]) #include self
        localvectorsz.append(sgn*tiltvectors[nn][i][2]) #include self
        for j in range(len(neigh[nn][i])):
            sgn=np.sign(tiltvectors[nn][neigh[nn][i][j]][2])
            localvectorsx.append(sgn*tiltvectors[nn][neigh[nn][i][j]][0])
            localvectorsy.append(sgn*tiltvectors[nn][neigh[nn][i][j]][1])
            localvectorsz.append(sgn*tiltvectors[nn][neigh[nn][i][j]][2])
        localvector[nn][i]=[np.mean(localvectorsx),np.mean(localvectorsy),np.mean(localvectorsz)]
       

In [23]:
# find angle between orientation vector and local average orientation vector for each lipid
# in range [0,180]
diffangle=list([] for _ in xrange(traj.n_frames))
for nn in range(traj.n_frames):
    diffangle[nn]=list([] for _ in xrange(len(allhead)))
    for i in range(len(allhead)):
        normlv=np.linalg.norm(localvector[nn][i])
        normtv=np.linalg.norm(tiltvectors[nn][i])
        cos=np.dot(localvector[nn][i],tiltvectors[nn][i])/(normlv*normtv)
        if (cos==0):
            diffangle[nn][i]=90
        elif (cos==1 and np.sign(localvector[nn][i][2])==np.sign(tiltvectors[nn][i][2])):
            diffangle[nn][i]=0
        elif (cos==1 and np.sign(localvector[nn][i][2])==np.sign(tiltvectors[nn][i][2])):
            diffangle[nn][i]=180
        else:
            diffangle[nn][i]=np.arccos(cos)*180./np.pi


In [24]:
#have array with values placing each head in one leaflet: 0=lower, 1=upper, 2=between
head_leaflet=list([] for _ in xrange(traj.n_frames))
for nn in range(traj.n_frames):
    head_leaflet[nn]=list([] for _ in xrange(len(allhead)))
    for i in range(len(allhead)):
        if (diffangle[nn][i]>120):
            head_leaflet[nn][i]=0
        elif (diffangle[nn][i]<60):
            head_leaflet[nn][i]=1
        else:
            head_leaflet[nn][i]=2

In [25]:
# want to identify all midplane chol and remove all "midplane" phospholipids
# tilt angle insufficient
# chol: find distance from ROH to nearest PO4; if beyond threshold, then in midplane; 
# else, copy leaflet ID (do for all phospholipids); only if copying non-midplane

for nn in range(traj.n_frames):
    fixedchol=0
    fixedphos=0
    midplanechol=0
    for i in range(len(allhead)):
        mindist=100.0
        if (head_leaflet[nn][i]==2 and (not resnames[allhead[i]]=='CHOL')): #phospholipids, must assign
            for j in range(len(neigh[nn][i])):
                if (not resnames[allhead[neigh[nn][i][j]]]=='CHOL'):
                    dist=np.linalg.norm(traj.xyz[nn][allhead[i]]-traj.xyz[nn][allhead[neigh[nn][i][j]]])
                    if (dist < mindist and (not head_leaflet[nn][neigh[nn][i][j]]==2)):
                        mindist=dist
                        correct_leaflet=head_leaflet[nn][neigh[nn][i][j]]
            head_leaflet[nn][i]=correct_leaflet
            fixedphos=fixedphos+1
    #must fix all phospholipids before all chols, since some may use fixed phospholipid leaflet id
    # so, must go through allhead twice
    for i in range(len(allhead)):
        mindist=100.0
        if (resnames[allhead[i]]=='CHOL'): #all chol, ignoring angle
            for j in range(len(neigh[nn][i])):
                if (not resnames[allhead[neigh[nn][i][j]]]=='CHOL'):
                    dist=np.linalg.norm(traj.xyz[nn][allhead[i]]-traj.xyz[nn][allhead[neigh[nn][i][j]]])
                    if (dist < mindist and (not head_leaflet[nn][neigh[nn][i][j]]==2)):
                        mindist=dist
                        correct_leaflet=head_leaflet[nn][neigh[nn][i][j]]
            if (mindist<1.4): #close to a headgroup, so in a leaflet
                head_leaflet[nn][i]=correct_leaflet
                fixedchol=fixedchol+1
            else:
                head_leaflet[nn][i]=2
                midplanechol=midplanechol+1

In [26]:
# only need upper leaflet heads for this situation
upperheads=list([] for _ in xrange(traj.n_frames))
for nn in range(traj.n_frames):
#    upperheads[nn]=[]
    for i in range(len(allhead)):
        if head_leaflet[nn][i]==1:
            upperheads[nn].append(allhead[i])

In [27]:
# only need upper leaflet heads for this situation
lowerheads=list([] for _ in xrange(traj.n_frames))
for nn in range(traj.n_frames):
    for i in range(len(allhead)):
        if head_leaflet[nn][i]==0:
            lowerheads[nn].append(allhead[i])

In [28]:
#x,y coordinates of outer for Voronoi
headxy=list([] for _ in xrange(traj.n_frames)) #[frame][head]
reg=list([] for _ in xrange(traj.n_frames))
frac_reg=list([] for _ in xrange(traj.n_frames))
vneigh=list([] for _ in xrange(traj.n_frames))
vneighres=list([] for _ in xrange(traj.n_frames))
for nn in range(traj.n_frames):
    headxy[nn]=list([] for _ in xrange(len(lowerheads[nn])))
    for i in range(len(lowerheads[nn])):
        lowerheadsi=lowerheads[nn][i]
        headxy[nn][i]=(traj.xyz[nn][lowerheadsi][0:2:1])
        

    vor=scipy.spatial.Voronoi(np.array(headxy[nn]))

#Voronoi neighbors vneigh

#generate list of neighbors, edgeneigh
    edgeneigh=list([] for _ in xrange(len(lowerheads[nn])))
    for k in range(len(vor.ridge_points)):
        pair=vor.ridge_points[k]
        edgeneigh[pair[0]].append(pair[1])
        edgeneigh[pair[1]].append(pair[0])



    vneigh[nn]=list([] for _ in xrange(len(lowerheads[nn])))  #lipid type
    vneighres[nn]=list([] for _ in xrange(len(lowerheads[nn])))
    for i in range(len(lowerheads[nn])):
        vneighheads=edgeneigh[i] #upperhead index, not atom
        if not vneighheads:
            vneigh[nn][i]=[]
            vneighres[nn][i]=[]
        else:
            for j in range(len(vneighheads)):
                vneigh[nn][i].append(resnames[lowerheads[nn][vneighheads[j]]])
                vneighres[nn][i].append(vneighheads[j])


In [29]:
lowerheadxy=list([] for _ in xrange(traj.n_frames))
lowerheadz=list([] for _ in xrange(traj.n_frames))

for nn in range(traj.n_frames):
    for i in range(len(lowerheads[nn])):
        if lowerheads[nn][i] in allhead:
            index = np.where(allhead==lowerheads[nn][i])
            lowerheadxy[nn].append(allheadxy[0][int(index[0])])
            lowerheadz[nn].append(allheadz[0][int(index[0])])

            
    

In [30]:
#only care about DBPC's neighbors
DBPC_vneighlist=list([] for _ in xrange(traj.n_frames))

for nn in range(traj.n_frames):
    for i in range(len(vneighres[nn])):
            if resnames[lowerheads[nn][i]]== "POPE":
                DBPC_vneighlist[nn].append([i]+vneighres[nn][i]) 
#for nn in range(traj.n_frames): 

  #  for i in range(len(DBPC_vneighlist[nn])):
       #    del DBPC_vneighlist[nn][i][2:]
                

In [31]:
membrane_normal=list([] for x in range(traj.n_frames))
for nn in range(traj.n_frames):
    for i in range(len(DBPC_vneighlist[nn])):
        point1=np.array([lowerheadxy[nn][DBPC_vneighlist[nn][i][0]][0],lowerheadxy[nn][DBPC_vneighlist[nn][i][0]][1],lowerheadz[nn][DBPC_vneighlist[nn][i][0]]])
        point2=np.array([lowerheadxy[nn][DBPC_vneighlist[nn][i][1]][0],lowerheadxy[nn][DBPC_vneighlist[nn][i][1]][1],lowerheadz[nn][DBPC_vneighlist[nn][i][0]]])
        point3=np.array([lowerheadxy[nn][DBPC_vneighlist[nn][i][2]][0],lowerheadxy[nn][DBPC_vneighlist[nn][i][2]][1],lowerheadz[nn][DBPC_vneighlist[nn][i][0]]])
        a=find_plane(point1,point2,point3)
        #print(point1,point2,point3)
        if math.isnan(a[0]):
            print(nn,i)
        membrane_normal[nn].append(a)

for nn in range(traj.n_frames):
    for i in range(len(membrane_normal[nn])):
        if membrane_normal[nn][i][2]>0:
            membrane_normal[nn][i][0]=(-1)*membrane_normal[nn][i][0]
            membrane_normal[nn][i][1]=(-1)*membrane_normal[nn][i][1]
            membrane_normal[nn][i][2]=(-1)*membrane_normal[nn][i][2]




In [32]:
for nn in range(traj.n_frames):
    for i in range(len(membrane_normal[nn])): 
        if math.isnan(membrane_normal[nn][i][0]):
            print(nn,i)

In [33]:
POPE_in_lowerleaflet = list([] for _ in xrange(traj.n_frames))

for nn in range(traj.n_frames):
    for i in range(len(lowerheads[nn])):
        if resnames[lowerheads[nn][i]]=='POPE':
            POPE_in_lowerleaflet[nn].append(lowerheads[nn][i])

In [72]:
len(lowerheads[10])

2689

In [71]:
len(lowerheadxy[10])

2689

# Find POPE Across from Lo/Ld

In [34]:
#count DPPC as fraction of upper total, to find global concentration of DPPC
upperdppc=list([] for _ in xrange(traj.n_frames))
#uppertotal=list([] for _ in xrange(traj.n_frames))
globalconc=list([] for _ in xrange(traj.n_frames))

for nn in range(traj.n_frames):
    upperdppc[nn]=0
    for j in range(len(upperheads[nn])):
        if(resnames[upperheads[nn][j]]=='DXPC' or resnames[upperheads[nn][j]]=='DBPC' or resnames[upperheads[nn][j]]=='DPPC' or resnames[upperheads[nn][j]]=='CHOL'):
            upperdppc[nn]=upperdppc[nn]+1
    globalconc[nn]=float(upperdppc[nn])/len(upperheads[nn])

In [35]:
#x,y coordinates of outer for Voronoi
headxy=list([] for _ in xrange(traj.n_frames)) #[frame][head]
reg=list([] for _ in xrange(traj.n_frames))
frac_reg=list([] for _ in xrange(traj.n_frames))
for nn in range(traj.n_frames):
#    headxy[nn]=list([] for _ in xrange(len(upperheads[nn])))
    for i in range(len(upperheads[nn])):
        upperheadsi=upperheads[nn][i]
        headxy[nn].append(traj.xyz[nn][upperheadsi][0:2:1])
        

    vor=scipy.spatial.Voronoi(np.array(headxy[nn]))

#Voronoi neighbors vneigh

#generate list of neighbors, edgeneigh
    edgeneigh=list([] for _ in xrange(len(upperheads[nn])))
    for k in range(len(vor.ridge_points)):
        pair=vor.ridge_points[k]
        edgeneigh[pair[0]].append(pair[1])
        edgeneigh[pair[1]].append(pair[0])



    vneigh=list([] for _ in xrange(len(upperheads[nn])))  #lipid type
    vneighres=list([] for _ in xrange(len(upperheads[nn])))
    for i in range(len(upperheads[nn])):
        vneighheads=edgeneigh[i] #upperhead index, not atom
        if not vneighheads:
            vneigh[i]=[]
            vneighres[i]=[]
        else:
            for j in range(len(vneighheads)):
                vneigh[i].append(resnames[upperheads[nn][vneighheads[j]]])
                vneighres[i].append(vneighheads[j])


#phase based on edges
    vphase=list([] for _ in xrange(len(upperheads[nn])))
    for i in range(len(upperheads[nn])):
        vneightot=1+len(vneigh[i]) #include self
        num_sat=0
        if (resnames[upperheads[nn][j]]=='DXPC' or resnames[upperheads[nn][j]]=='DBPC' or resnames[upperheads[nn][j]]=='DPPC' or resnames[upperheads[nn][j]]=='CHOL'):
            num_sat=num_sat+1
        for j in range(len(vneigh[i])):
            if (vneigh[i][j]=='DXPC' or vneigh[i][j]=='DBPC' or vneigh[i][j]=='DPPC' or vneigh[i][j]=='CHOL'):
                num_sat=num_sat+1
        vlocal_conc=float(num_sat)/vneightot #integer would always be zero
        if (vlocal_conc > globalconc[nn]):
            vphase[i] = 1 # Lo=1
        else:
            vphase[i] = 0 # Ld=0
        
#iterate edge-based phase determination to reduce small clusters
    vphase2=list([] for _ in xrange(len(upperheads[nn])))
    for i in range(len(upperheads[nn])):
        vneightot=1+len(vneigh[i])
        num_lo=0
        if (vphase[i]==1):
            num_lo=num_lo+1
        for j in range(len(vneigh[i])):
            if(vphase[vneighres[i][j]]==1):
                num_lo=num_lo+1
        vlocal_conc=float(num_lo)/vneightot
        if (vlocal_conc > 0.49):
            vphase2[i]=1
        else:
            vphase2[i]=0


#iterate edge-based phase determination to reduce small clusters
    vphase3=list([] for _ in xrange(len(upperheads[nn])))
    for i in range(len(upperheads[nn])):
        vneightot=1+len(vneigh[i])
        num_lo=0
        if (vphase2[i]==1):
            num_lo=num_lo+1
        for j in range(len(vneigh[i])):
            if(vphase2[vneighres[i][j]]==1):
                num_lo=num_lo+1
        vlocal_conc=float(num_lo)/vneightot
        if (vlocal_conc > 0.49):
            vphase3[i]=1
        else:
            vphase3[i]=0
            
            
# calculate opposite phase
    oppneigh=list([] for _ in xrange(len(upperheads[nn]))) #for upper, lower neighbors (xy dist)
    for i in range(len(upperheads[nn])):
        for j in range(len(neigh[nn][i])):
            if (head_leaflet[nn][neigh[nn][i][j]]==0): # if lower
                oppneigh[i].append(resnames[allhead[neigh[nn][i][j]]])
#           oppneighres[i].append(allhead[neigh[i][j]])

    oppphase=list([] for _ in xrange(len(upperheads[nn])))
    for i in range(len(upperheads[nn])):
        oppneightot=len(oppneigh[i])
        num_sat=0
        for j in range(len(oppneigh[i])):
            if (oppneigh[i][j]=='DXPC' or oppneigh[i][j]=='DBPC' or oppneigh[i][j]=='DPPC' or oppneigh[i][j]=='CHOL'):
                num_sat=num_sat+1
        if (oppneightot):
            local_conc=float(num_sat)/oppneightot
        else:
            local_conc=0
        if (local_conc > globalconc[nn]):
            oppphase[i] = 1 #Lo
        else:
            oppphase[i] = 0 #Ld
    

# for more logical comparison, iterate twice, using upper Voronoi neighbors' oppphase values
    oppphase2=list([] for _ in xrange(len(upperheads[nn])))
    for i in range(len(upperheads[nn])):
        vneightot=1+len(vneigh[i]) #include self, Voronoi same-leaflet neighbors
        num_lo=0
        if (oppphase[i]==0):
            num_lo=num_lo+1
        for j in range(len(vneigh[i])):
            if (oppphase[vneighres[i][j]]==1):
                num_lo=num_lo+1
        local_conc=float(num_lo)/vneightot
        if (local_conc > 0.49):
            oppphase2[i]=1
        else: 
            oppphase2[i]=0

    oppphase3=list([] for _ in xrange(len(upperheads[nn])))
    for i in range(len(upperheads[nn])):
        vneightot=1+len(vneigh[i]) #include self, Voronoi same-leaflet neighbors
        num_lo=0
        if (oppphase2[i]==0):
            num_lo=num_lo+1
        for j in range(len(vneigh[i])):
            if (oppphase2[vneighres[i][j]]==1):
                num_lo=num_lo+1
        local_conc=float(num_lo)/vneightot
        if (local_conc > 0.49):
            oppphase3[i]=1
        else: 
            oppphase3[i]=0
        
        
# compare oppphase3 to vphase3 to check for registration
    reg[nn]=list([] for _ in xrange(len(upperheads[nn])))
    for i in range(len(upperheads[nn])):
        if (oppphase3[i]==vphase3[i]):
            reg[nn][i]=1 # registered
        else:
            reg[nn][i]=0 # not registered
    frac_reg[nn]=float(sum(reg[nn]))/len(reg[nn])

In [36]:
#x,y coordinates of outer for Voronoi
headxy=list([] for _ in xrange(traj.n_frames)) #[frame][head]
intmember=list([] for _ in xrange(traj.n_frames))
intfrac=list([] for _ in xrange(traj.n_frames))
regintfrac=list([] for _ in xrange(traj.n_frames))
u_phase_global=list([] for _ in xrange(traj.n_frames))
for nn in range(traj.n_frames):
#    headxy[nn]=list([] for _ in xrange(len(upperheads[nn])))
    for i in range(len(upperheads[nn])):
        upperheadsi=upperheads[nn][i]
        headxy[nn].append(traj.xyz[nn][upperheadsi][0:2:1])
        

    vor=scipy.spatial.Voronoi(np.array(headxy[nn]))

#Voronoi neighbors vneigh

#generate list of neighbors, edgeneigh
    edgeneigh=list([] for _ in xrange(len(upperheads[nn])))
    for k in range(len(vor.ridge_points)):
        pair=vor.ridge_points[k]
        edgeneigh[pair[0]].append(pair[1])
        edgeneigh[pair[1]].append(pair[0])



    vneigh=list([] for _ in xrange(len(upperheads[nn])))  #lipid type
    vneighres=list([] for _ in xrange(len(upperheads[nn])))
    for i in range(len(upperheads[nn])):
        vneighheads=edgeneigh[i] #upperhead index, not atom
        if not vneighheads:
            vneigh[i]=[]
            vneighres[i]=[]
        else:
            for j in range(len(vneighheads)):
                vneigh[i].append(resnames[upperheads[nn][vneighheads[j]]])
                vneighres[i].append(vneighheads[j])


#phase based on edges
    vphase=list([] for _ in xrange(len(upperheads[nn])))
    for i in range(len(upperheads[nn])):
        vneightot=1+len(vneigh[i]) #include self
        num_sat=0
        if (resnames[upperheads[nn][j]]=='DXPC' or resnames[upperheads[nn][j]]=='DBPC' or resnames[upperheads[nn][j]]=='DPPC' or resnames[upperheads[nn][j]]=='CHOL'):
            num_sat=num_sat+1
        for j in range(len(vneigh[i])):
            if (vneigh[i][j]=='DXPC' or vneigh[i][j]=='DBPC' or vneigh[i][j]=='DPPC' or vneigh[i][j]=='CHOL'):
                num_sat=num_sat+1
        vlocal_conc=float(num_sat)/vneightot #integer would always be zero
        if (vlocal_conc > globalconc[nn]):
            vphase[i] = 1 # Lo=1
        else:
            vphase[i] = 0 # Ld=0
        
#iterate edge-based phase determination to reduce small clusters
    vphase2=list([] for _ in xrange(len(upperheads[nn])))
    for i in range(len(upperheads[nn])):
        vneightot=1+len(vneigh[i])
        num_lo=0
        if (vphase[i]==1):
            num_lo=num_lo+1
        for j in range(len(vneigh[i])):
            if(vphase[vneighres[i][j]]==1):
                num_lo=num_lo+1
        vlocal_conc=float(num_lo)/vneightot
        if (vlocal_conc > 0.49):
            vphase2[i]=1
        else:
            vphase2[i]=0


#iterate edge-based phase determination to reduce small clusters
    vphase3=list([] for _ in xrange(len(upperheads[nn])))
    for i in range(len(upperheads[nn])):
        vneightot=1+len(vneigh[i])
        num_lo=0
        if (vphase2[i]==1):
            num_lo=num_lo+1
        for j in range(len(vneigh[i])):
            if(vphase2[vneighres[i][j]]==1):
                num_lo=num_lo+1
        vlocal_conc=float(num_lo)/vneightot
        if (vlocal_conc > 0.49):
            vphase3[i]=1
        else:
            vphase3[i]=0
            
#################################################################################
#ONLY GOING TO USE VPHASE TO CREAT U_PHASE
    domain=list([] for _ in xrange(len(upperheads[nn])))
    for i in range(len(upperheads[nn])): #N: Cycling through all molecule heads
        domain[i]=0 # 0 = unassigned; N: Assigning each domain a value of 0

    recurlist=[] # list of heads that must run through domain algorithm because in chain of neighbors

    domain[0]=1 # first lipid sets first domain
    for i in range(len(vneighres[0])): # vneighres is allheadz index, not atom
        ni=vneighres[0][i] #N: Looking at specific patch
        if vphase[ni]==vphase[0]: #only neighbors in same phase; N: Check if they have the same phase
            domain[ni]=1
            recurlist.append(ni) #N: Add it the heads that need to be checked against their neighbors
    num_domains=1
     
    while len(recurlist): # no new domain should be introduced in this section
        headi=recurlist[0] #N: Looking at the first entry in recurlist
        neighi=vneighres[headi] #N: Finding its neighbors
        recurlist=recurlist[1:] # remove 1st element, now being handled
        for j in range(len(neighi)):#N: For all of its neighbors
            ni=neighi[j] 
            if (vphase[ni]==vphase[headi] and domain[ni]==0): #avoid infinitely putting back into recurlist; N: is the phase same as domian
                domain[ni]=domain[headi] #N: Domain is the same as the head
                recurlist.append(ni) #N: Put it back in the list to be checked again

    #now iterate over other lipids, excluding those already marked
    for i in range(len(upperheads[nn])-1): # 0 already done
        if domain[i+1]==0: #not yet marked
            neighi=vneighres[i+1]
            num_domains=num_domains+1
            domain[i+1]=num_domains
            for j in range(len(neighi)):
                ni=neighi[j]
                if vphase[ni]==vphase[i+1]:
                    domain[ni]=domain[i+1] 
                    recurlist.append(ni)
            while len(recurlist): #until exhausted, then move onto next new domain
                headi=recurlist[0]
                neighi=vneighres[headi]
                recurlist=recurlist[1:]
                for j in range(len(neighi)):
                    ni=neighi[j]
                    if (vphase[ni]==vphase[headi] and domain[ni]==0):
                        domain[ni]=domain[headi]
                        recurlist.append(ni)
                        
    # see how many lipids in each domain
    num_domains=max(domain)
    domainsize=list([] for _ in xrange(num_domains))
    for i in range(len(domainsize)):
        domainsize[i]=0
    for i in range(len(domain)):
        di=domain[i]-1 #return to 0-index
        domainsize[di]=domainsize[di]+1

    #N: Method to get rid of domains that are less than 10 lipids long. 1) Find domains which are less than 10 lipids 
    #N: 2) Take every lipid with a small domain and add it to a list 3) Take that list and change the phase, and make
    #N: an  updated phase list

    #N: Going to make a list of the domains which are too small
    small_domain=[]
    lipid_counter=0 #N: Total number of lipids 
    for i in range(len(domainsize)): 
        if domainsize[i]<10: 
            small_domain.append(i+1)
            lipid_counter=lipid_counter+domainsize[i]


    #N: Go through domain(holds which domain every lipid belongs to) and see if its domain is in small_domian
    #N: If it is, its phase needs to be changed 
    phase_req_change=[]
    for i in range(len(domain)):
        if domain[i] in small_domain:
            phase_req_change.append(i)

    #N: Go through phase_req_change. Each entry is the lipid whose phase needs to be switched. So take that entry
    #N: and plug it into u_phase. Then switch the value of u_phase.
    u_phase=[]

    for i in range(len(vphase)):
        if i in phase_req_change:
            #print(n)
            if vphase[i]==0:
                u_phase.append(1)
            if vphase[i]==1:
                u_phase.append(0)
        else: 
            u_phase.append(vphase[i])
    u_phase_global[nn]=u_phase


In [37]:
bottom_oppneigh=list([] for _ in xrange(traj.n_frames)) #for upper, lower neighbors (xy dist)
for nn in range(traj.n_frames):
    bottom_oppneigh[nn]=list([] for _ in xrange(len(lowerheads[nn]))) #for upper, lower neighbors (xy dist)
    for i in range(len(lowerheads[nn])):
        for j in range(len(neigh[nn][i])):
            if (head_leaflet[nn][neigh[nn][i][j]]==1): # if upper
                bottom_oppneigh[nn][i].append(allhead[neigh[nn][i][j]])


In [38]:
bottom_acrossLO=list([] for _ in xrange(traj.n_frames))
bottom_acrossLD = list([] for _ in xrange(traj.n_frames))
for nn in range(traj.n_frames):

    for i in range(len(bottom_oppneigh[nn])):
        s=0
        for j in range(len(bottom_oppneigh[nn][i])):
            s=np.mean(int(u_phase_global[nn][upperheads[nn].index(bottom_oppneigh[nn][i][j])]))
        if float(s)>=.5:
            bottom_acrossLO[nn].append(lowerheads[nn][i])
        if float(s)<.5:
            bottom_acrossLD[nn].append(lowerheads[nn][i])
        

# Split Desired Lipid into Chains - only need sn1

In [301]:
DBPCchunked=list([] for _ in xrange(traj.n_frames))
sn1=list([] for _ in xrange(traj.n_frames))
LO_LD=list([] for _ in xrange(traj.n_frames))
sn2=list([] for _ in xrange(traj.n_frames))
sn3=list([] for _ in xrange(traj.n_frames))

for nn in range(traj.n_frames): 
    DBPCchunked[nn]=[DBPCs[i:i + 52] for i in xrange(0, len(DBPCs), 52)]

for nn in range(traj.n_frames):
    for i in range(len(DBPCchunked[nn])):
        if DBPCchunked[nn][i][7] in POPE_in_lowerleaflet[nn]:
            a=DBPCchunked[nn][i][14:32]
            b=DBPCchunked[nn][i][33:50] #double-bondchain
            #print(a,b)
            a=np.delete(a,1)
            b=np.delete(b,1)

            #a.append(DBPCchunked[nn][i][50:52])
            sn1[nn].append(a)
            sn2[nn].append(b)
            sn3[nn].append(b)
for nn in range(traj.n_frames):
    for i in range(len(DBPCchunked[nn])):
        if DBPCchunked[nn][i][7] in POPE_in_lowerleaflet[nn]: 
            if DBPCchunked[nn][i][7] in bottom_acrossLO[nn]:
                LO_LD[nn].append(1)
                
            if DBPCchunked[nn][i][7] in bottom_acrossLD[nn]:
                LO_LD[nn].append(0)

curv = list([] for _ in xrange(traj.n_frames))
for nn in range(traj.n_frames):
    for i in range(len(DBPCchunked[nn])):
        if DBPCchunked[nn][i][7] in POPE_in_lowerleaflet[nn]: 
            x = lowerheadxy[nn][lowerheads[nn].index(DBPCchunked[nn][i][7])][1]
            if x < float(2.5):
                curv[nn].append(0)
            if x > float(15.0) and x < float(17.5):
                curv[nn].append(1)
            if x > float(2.5) and x < float(15.0):
                curv[nn].append(2)
            if x > float(17.5):
                curv[nn].append(2)
            
            

# Vector Calculation

In [107]:
localz = list([] for _ in xrange(traj.n_frames))

for nn in range(traj.n_frames):
    for i in range(len(sn2[nn])):
        
        for j in range(len(sn2[nn][i])-2):
            #print(nn,i,j)
            a=sn2[nn][i][j]-sn2[nn][0][0]+33
            b=sn2[nn][i][j+2]-sn2[nn][0][0]+33
            #print(a,b)
            vector=local_vector(lipidheadxyz[nn][a], lipidheadxyz[nn][b])
            #if vector[0] and vector[1] and vector[2] <0:
                #vector=vector*-1
            localz[nn].append(vector)
            

In [108]:
localx = list([] for _ in xrange(traj.n_frames))

for nn in range(traj.n_frames):
    for i in range(len(localz[nn])):
        localx[nn].append(perpendicular_vector(localz[nn][i]))

In [109]:
localy = list([] for _ in xrange(traj.n_frames))

for nn in range(traj.n_frames):
    for i in range(len(localx[nn])): 
        n = np.cross(localz[nn][i],localx[nn][i])
        n=n/magvect(n)
        if n[0]  <0:
                n=n*-1
        localy[nn].append(n)
     

In [110]:
localx_lipid = list([] for _ in xrange(traj.n_frames))
localy_lipid = list([] for _ in xrange(traj.n_frames))

for nn in range(traj.n_frames):
    localx_lipid[nn]=[localx[nn][i:i + 14] for i in xrange(0, len(localx[nn]), 14)]
    localy_lipid[nn]=[localy[nn][i:i + 14] for i in xrange(0, len(localy[nn]), 14)]

# Order Parameter Calculation

In [111]:
Anglexx = list([] for _ in xrange(traj.n_frames))
Angleyy = list([] for _ in xrange(traj.n_frames))

for nn in range(traj.n_frames):
    for i in range(len(localx_lipid[nn])):
        for j in range(len(localx_lipid[nn][i])):
            a=cos_angle(localx_lipid[nn][i][j].tolist(),membrane_normal[nn][i])
            if math.isnan(a)==True:
                print(a)
            if a>np.pi/2:
                a=float(np.pi - a)
            Anglexx[nn].append(a)
            
for nn in range(traj.n_frames):
    for i in range(len(localy_lipid[nn])):
        for j in range(len(localy_lipid[nn][i])):
            a=cos_angle(localy_lipid[nn][i][j],membrane_normal[nn][i])
            if math.isnan(a)==True:     
                print(localy_lipid[nn][i][j],membrane_normal[nn][j])
            if a>np.pi/2:
                
                a=float(np.pi - a)
            Angleyy[nn].append(a)


In [112]:
Sxx = list([] for _ in xrange(traj.n_frames))
Syy = list([] for _ in xrange(traj.n_frames))
Sxy = list([] for _ in xrange(traj.n_frames))

for nn in range(traj.n_frames):
    for i in range(len(localx_lipid[nn])):
        for j in range(len(localx_lipid[nn][i])):
            Sxx[nn].append((3*cos_angle(localx_lipid[nn][i][j],membrane_normal[nn][i])**2 - 1)/2)
            Syy[nn].append((3*cos_angle(localy_lipid[nn][i][j],membrane_normal[nn][i])**2 - 1)/2)


In [46]:
OrderParam = list([] for _ in xrange(traj.n_frames))
FinalOP = list([] for _ in xrange(traj.n_frames))
for nn in range(traj.n_frames):
    for i in range(len(Sxx[nn])):
        OrderParam[nn].append((2*Sxx[nn][i]+Syy[nn][i])/3)
for nn in range(traj.n_frames): 
    FinalOP[nn]=[OrderParam[nn][i:i + 14] for i in xrange(0, len(OrderParam[nn]), 14)]

In [121]:
len(FinalOP[0])

876

In [122]:
len(LO_LD[0])

876

In [307]:
FinalOP_LO=list([] for _ in xrange(traj.n_frames))
FinalOP_LD=list([] for _ in xrange(traj.n_frames))

for nn in range(traj.n_frames):
    for i in range(len(FinalOP[nn])):
        if curv[nn][i] == 0:
            FinalOP_LO[nn].append(FinalOP[nn][i])
        if curv[nn][i] == 1:
            FinalOP_LD[nn].append(FinalOP[nn][i])
            
            

In [308]:
DataOP_LO = list([] for _ in xrange(traj.n_frames))
DataOP_LD = list([] for _ in xrange(traj.n_frames))


for nn in range(traj.n_frames):
    #print(nn)
    DataOP_LO[nn]=np.mean(FinalOP_LO[nn],axis=0,dtype=float)


Final_AVG_LO=np.mean(DataOP_LO,axis=0,dtype=float)
Final_STD_LO=np.std(DataOP_LO,axis=0,dtype=float)

DataOP_LD = list([] for _ in xrange(traj.n_frames))

for nn in range(traj.n_frames):
    DataOP_LD[nn]=np.mean(FinalOP_LD[nn],axis=0,dtype=float)

Final_AVG_LD=np.mean(DataOP_LD,axis=0,dtype=float)
Final_STD_LD=np.std(DataOP_LD,axis=0,dtype=float)

In [311]:
(Final_AVG_LD)

array([-0.21422952, -0.24148338, -0.24308334, -0.24153058, -0.26291712,
       -0.25176286, -0.23692889, -0.23296074, -0.21439995, -0.19958726,
       -0.17849036, -0.14687031, -0.09915779, -0.0605841 ])

In [312]:
Final_AVG_LO

array([-0.21687741, -0.24739859, -0.28190972, -0.2988986 , -0.29719723,
       -0.28995557, -0.28554574, -0.27312678, -0.27411087, -0.24829348,
       -0.21446304, -0.18079302, -0.13774933, -0.09551836])

In [3037]:
file=open('aSymmetric-POPE-OP-LO-AVG.txt',"a+")

file.write(str(float(name)))
file.write(",")

for i in range(len(Final_AVG_LO)):
    file.write(str(float(Final_AVG_LO[i])))
    file.write(",")
               
file.write('\n')
file.write('\n')

In [3038]:
file=open('aSymmetric-POPE-OP-LO-STD.txt',"a+")

file.write(str(float(name)))
file.write(",")
for i in range(len(Final_STD_LO)):
    file.write(str(float(Final_STD_LO[i])))
    file.write(",")
file.write('\n')
file.write('\n')

In [3039]:
file=open('aSymmetric-POPE-OP-LD-AVG.txt',"a+")

file.write(str(float(name)))
file.write(",")

for i in range(len(Final_AVG_LD)):
    file.write(str(float(Final_AVG_LD[i])))
    file.write(",")
               
file.write('\n')
file.write('\n')

In [3041]:
file=open('aSymmetric-POPE-OP-LD-STD.txt',"a+")

file.write(str(float(name)))
file.write(",")
for i in range(len(Final_STD_LD)):
    file.write(str(float(Final_STD_LD[i])))
    file.write(",")
file.write('\n')
file.write('\n')